In [3]:
# ===============================================================
# 🧠 U23 Player Grading System — Multi-League (Local Edition)
# ---------------------------------------------------------------
# • Calculates 20–80 grades per league
# • Uses "Team within selected timeframe"
# • Filters: U23, non–Big 5, ≥900 minutes
# • Excel tabs:
#     1️⃣ OFP  (Top 15% by OFP_20_80)
#     2️⃣ Impact (Top 15% by Impact_20_80)
#     3️⃣ Top 50 OFP
#     4️⃣ Top 50 Impact
# • Creates PNG visuals for Top 25 OFP & Impact
#   (labels as "Player [Team within selected timeframe]")
# ===============================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from openpyxl import load_workbook
from openpyxl.formatting.rule import ColorScaleRule
import glob, os

# ---------- Folder Settings ----------
FOLDER_PATH = r"/Users/user/Downloads/Mid America/"   # 👈 update your folder path
OUTPUT_PATH = os.path.join(FOLDER_PATH, "U23 Mid America Reports.xlsx")

# ---------- Helper Functions ----------
def standardize_columns(df):
    df = df.copy()
    df.columns = (
        df.columns.astype(str)
        .str.strip()
        .str.replace(r"[/%]", " pct ", regex=True)
        .str.replace(r"[.,]", " ", regex=True)
        .str.replace(r"\s+", "_", regex=True)
        .str.lower()
    )
    return df

def safe_numeric(series):
    return pd.to_numeric(series, errors="coerce")

def league_percentile_scale(s, lo=10, hi=90):
    s = safe_numeric(s)
    if s.dropna().empty:
        return pd.Series(np.nan, index=s.index)
    p_lo, p_hi = np.nanpercentile(s.dropna(), [lo, hi])
    def scale(x):
        if pd.isna(x): return np.nan
        if x <= p_lo: return 20
        if x >= p_hi: return 80
        return 20 + (x - p_lo) * (60 / (p_hi - p_lo))
    return s.apply(scale)

def weighted_mean(series_dict):
    idx = next(iter(series_dict.values()))[0].index
    val_sum = np.zeros(len(idx))
    w_sum = np.zeros(len(idx))
    for s, w in series_dict.values():
        s = safe_numeric(s)
        valid = ~s.isna()
        val_sum[valid] += s[valid] * w
        w_sum[valid] += w
    return pd.Series(
        np.divide(val_sum, w_sum, out=np.full_like(val_sum, np.nan), where=w_sum>0),
        index=idx,
    )

def detect_position_group(p):
    if not isinstance(p, str): return "OUTFIELD"
    p = p.upper()
    if "GK" in p: return "GK"
    if any(x in p for x in ["CB","LB","RB","DEF"]): return "DEF"
    if any(x in p for x in ["CM","DM","AM","MID","MF"]): return "MID"
    if any(x in p for x in ["FW","ST","CF","W","ATT"]): return "FWD"
    return "OUTFIELD"

def grade_meaning(score):
    if pd.isna(score): return ""
    if score >= 75: return "World-Class"
    elif score >= 65: return "Elite Starter"
    elif score >= 55: return "Strong Starter"
    elif score >= 45: return "Average Starter"
    elif score >= 35: return "Squad / Depth"
    elif score >= 25: return "Fringe / Replacement"
    else: return "Below Replacement"

# ---------- Core Function ----------
def build_grades(df, league_name):
    df = standardize_columns(df)

    # Use the correct team column
    if "team_within_selected_timeframe" in df.columns:
        df["team_name"] = df["team_within_selected_timeframe"]
    elif "team" in df.columns:
        df["team_name"] = df["team"]
    else:
        raise ValueError(f"{league_name}: Missing team column")

    if not all(col in df.columns for col in ["player", "team_name", "position", "age"]):
        raise ValueError(f"{league_name}: Missing core columns")

    pos_group = df["position"].apply(detect_position_group)

    TECH = weighted_mean({
        "pass_acc": (league_percentile_scale(df.get("accurate_passes_pct_")), 0.35),
        "prog_pass": (league_percentile_scale(df.get("progressive_passes_per_90")), 0.25),
        "prog_run": (league_percentile_scale(df.get("progressive_runs_per_90")), 0.15),
        "dribble_sr": (league_percentile_scale(df.get("successful_dribbles_pct_")), 0.15),
        "smart_pass": (league_percentile_scale(df.get("smart_passes_per_90")), 0.10),
    })

    TACT = weighted_mean({
        "xg": (league_percentile_scale(df.get("xg_per_90")), 0.35),
        "xa": (league_percentile_scale(df.get("xa_per_90")), 0.35),
        "key_pass": (league_percentile_scale(df.get("key_passes_per_90")), 0.20),
        "box_touches": (league_percentile_scale(df.get("touches_in_box_per_90")), 0.10),
    })

    PHYS = weighted_mean({
        "duels_vol": (league_percentile_scale(df.get("duels_per_90")), 0.20),
        "duels_win": (league_percentile_scale(df.get("duels_won_pct_")), 0.20),
        "aerials_vol": (league_percentile_scale(df.get("aerial_duels_per_90")), 0.15),
        "aerials_win": (league_percentile_scale(df.get("aerial_duels_won_pct_")), 0.15),
        "def_actions": (league_percentile_scale(df.get("successful_defensive_actions_per_90") + df.get("interceptions_per_90")), 0.20),
        "accels": (league_percentile_scale(df.get("accelerations_per_90")), 0.10),
    })

    PSYCH = weighted_mean({
        "few_fouls": (league_percentile_scale(-df.get("fouls_per_90")), 0.5),
        "few_yellows": (league_percentile_scale(-df.get("yellow_cards_per_90")), 0.3),
        "few_reds": (league_percentile_scale(-df.get("red_cards_per_90")), 0.2),
    })

    df["xContribution"] = (
        safe_numeric(df.get("xg_per_90"))
        + safe_numeric(df.get("xa_per_90"))
        + 0.5 * (safe_numeric(df.get("successful_defensive_actions_per_90")) + safe_numeric(df.get("interceptions_per_90")))
        + 0.1 * safe_numeric(df.get("duels_per_90"))
    )
    df["replacement"] = df.groupby(pos_group)["xContribution"].transform(lambda x: np.nanpercentile(x, 25))
    df["WAR"] = (df["xContribution"] - df["replacement"]) / 2.7

    if df["WAR"].dropna().shape[0] > 10:
        p5, p95 = np.nanpercentile(df["WAR"].dropna(), [5, 95])
        df["WAR_20_80"] = 20 + 60 * (df["WAR"] - p5) / (p95 - p5)
        df["WAR_20_80"] = df["WAR_20_80"].clip(20, 80)
    else:
        df["WAR_20_80"] = np.nan

    df["Impact_Score"] = (
        safe_numeric(df.get("xg_per_90")) + safe_numeric(df.get("xa_per_90"))
        + 0.2 * (safe_numeric(df.get("progressive_passes_per_90")) + safe_numeric(df.get("progressive_runs_per_90")))
        + 0.3 * (safe_numeric(df.get("successful_defensive_actions_per_90")) + safe_numeric(df.get("interceptions_per_90")))
        - 0.5 * (safe_numeric(df.get("fouls_per_90")) + safe_numeric(df.get("yellow_cards_per_90")) + safe_numeric(df.get("red_cards_per_90")))
    )
    df["Impact_20_80"] = league_percentile_scale(df["Impact_Score"])

    weights = {
        "FWD": dict(TECH=0.30, TACT=0.35, PHYS=0.20, PSYCH=0.10, WAR=0.05),
        "MID": dict(TECH=0.30, TACT=0.30, PHYS=0.20, PSYCH=0.10, WAR=0.10),
        "DEF": dict(TECH=0.25, TACT=0.25, PHYS=0.30, PSYCH=0.10, WAR=0.10),
        "GK":  dict(TECH=0.20, TACT=0.35, PHYS=0.25, PSYCH=0.10, WAR=0.10),
        "OUTFIELD": dict(TECH=0.30, TACT=0.35, PHYS=0.25, PSYCH=0.05, WAR=0.05)
    }

    OFP = []
    for i, pg in enumerate(pos_group):
        w = weights.get(pg, weights["OUTFIELD"])
        vals = {"TECH": TECH[i], "TACT": TACT[i],
                "PHYS": PHYS[i], "PSYCH": PSYCH[i],
                "WAR": df["WAR_20_80"].iloc[i]}
        num = sum(vals[k]*w[k] for k in w if not np.isnan(vals[k]))
        den = sum(w[k] for k in w if not np.isnan(vals[k]))
        OFP.append(num/den if den>0 else np.nan)

    grades = pd.DataFrame({
        "League": league_name,
        "Player": df["player"],
        "Team within selected timeframe": df["team_name"],
        "Age": safe_numeric(df["age"]),
        "Position": df["position"],
        "PosGroup": pos_group,
        "TECH_20_80": TECH.round(1),
        "TACT_20_80": TACT.round(1),
        "PHYS_20_80": PHYS.round(1),
        "PSYCH_20_80": PSYCH.round(1),
        "WAR": df["WAR"].round(3),
        "WAR_20_80": df["WAR_20_80"].round(1),
        "Impact_20_80": df["Impact_20_80"].round(1),
        "OFP_20_80": np.round(OFP, 1),
    })
    grades["Meaning"] = grades["OFP_20_80"].apply(grade_meaning)
    grades["Projected_OFP"] = np.round(grades["OFP_20_80"] + (24 - grades["Age"]) * 0.5, 1)

    if "minutes_played" in df.columns:
        grades["Minutes"] = safe_numeric(df["minutes_played"])
    elif "minutes" in df.columns:
        grades["Minutes"] = safe_numeric(df["minutes"])
    else:
        grades["Minutes"] = np.nan

    return grades

# ---------- Process All Excel Files ----------
all_grades = []
for file in glob.glob(os.path.join(FOLDER_PATH, "*.xlsx")):
    league = os.path.splitext(os.path.basename(file))[0]
    try:
        df = pd.read_excel(file)
        g = build_grades(df, league)
        all_grades.append(g)
        print(f"✅ {league}: {len(g)} players processed")
    except Exception as e:
        print(f"⚠️ {league}: error -> {e}")

# ---------- Combine & Filter ----------
combined = pd.concat(all_grades, ignore_index=True)
big5 = ["England", "France", "Italy", "Germany", "Spain"]
filtered = combined[
    (combined["Age"] <= 23)
    & (~combined["League"].isin(big5))
    & (combined["Minutes"] >= 900)
].copy()

# ---------- Rankings ----------
ofp_sorted = filtered.sort_values("OFP_20_80", ascending=False)
impact_sorted = filtered.sort_values("Impact_20_80", ascending=False)

top15_ofp = ofp_sorted.head(int(np.ceil(len(ofp_sorted) * 0.15)))
top15_impact = impact_sorted.head(int(np.ceil(len(impact_sorted) * 0.15)))
top50_ofp = ofp_sorted.head(50)
top50_impact = impact_sorted.head(50)

print(f"\n🌍 Non–Big 5 U23 (≥900 min): {len(filtered)}")
print(f"🏅 Top 15% OFP: {len(top15_ofp)} ⚡ Top 15% Impact: {len(top15_impact)}")

# ---------- Excel Save ----------
with pd.ExcelWriter(OUTPUT_PATH, engine="openpyxl") as writer:
    top15_ofp.to_excel(writer, sheet_name="OFP", index=False)
    top15_impact.to_excel(writer, sheet_name="Impact", index=False)
    top50_ofp.to_excel(writer, sheet_name="Top 50 OFP", index=False)
    top50_impact.to_excel(writer, sheet_name="Top 50 Impact", index=False)

# ---------- Add Heatmap ----------
wb = load_workbook(OUTPUT_PATH)
for sheet in ["OFP", "Impact", "Top 50 OFP", "Top 50 Impact"]:
    ws = wb[sheet]
    rule = ColorScaleRule(start_type='num', start_value=20, start_color="F8696B",
                          mid_type='num', mid_value=50, mid_color="FFEB84",
                          end_type='num', end_value=80, end_color="63BE7B")
    col_map = {c.value: i for i, c in enumerate(ws[1], 1)}
    for key in ["OFP_20_80", "Impact_20_80"]:
        if key in col_map:
            idx = col_map[key]
            ws.conditional_formatting.add(f"{chr(64+idx)}2:{chr(64+idx)}{ws.max_row}", rule)
wb.save(OUTPUT_PATH)
wb.close()

print(f"💾 Excel report saved → {OUTPUT_PATH}")

# ---------- PNG Visuals (Top 25) ----------
def make_barplot(df, col, title, filename):
    top = df.head(25)[["Player", "Team within selected timeframe", col]].copy()
    top["Label"] = top["Player"] + " [" + top["Team within selected timeframe"].astype(str) + "]"
    top = top.iloc[::-1]  # reverse order for readability
    plt.figure(figsize=(10, 8))
    plt.barh(top["Label"], top[col], color="royalblue", alpha=0.85)
    plt.xlabel(col)
    plt.title(title)
    plt.grid(alpha=0.3, axis="x")
    plt.tight_layout()
    out_path = os.path.join(FOLDER_PATH, filename)
    plt.savefig(out_path, dpi=300)
    plt.close()
    print(f"📊 Saved {filename}")

make_barplot(ofp_sorted, "OFP_20_80", "Top 25 U23 Players by OFP", "Top25_OFP.png")
make_barplot(impact_sorted, "Impact_20_80", "Top 25 U23 Players by Impact", "Top25_Impact.png")


✅ Honduras: 351 players processed
✅ Nicaragua: 294 players processed
⚠️ U23 Mid America Reports: error -> 'numpy.float64' object has no attribute 'dropna'
✅ Costa Rica: 286 players processed
✅ Guatemala: 347 players processed
✅ Panama: 451 players processed

🌍 Non–Big 5 U23 (≥900 min): 96
🏅 Top 15% OFP: 15 ⚡ Top 15% Impact: 15
💾 Excel report saved → /Users/user/Downloads/Mid America/U23 Mid America Reports.xlsx
📊 Saved Top25_OFP.png
📊 Saved Top25_Impact.png


In [2]:
# ---------- PROFESSIONAL PRESENTATION-STYLE PDF REPORT (FULL-BLEED VISUALS) ----------
from reportlab.lib import colors
from reportlab.lib.pagesizes import landscape, A4
from reportlab.platypus import (
    SimpleDocTemplate, Paragraph, Spacer, Image, Table, TableStyle, PageBreak
)
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch

# ---------- Paths ----------
PDF_PATH = os.path.join(FOLDER_PATH, "U23 Mid America.pdf")

# ---------- Styles ----------
styles = getSampleStyleSheet()
styles.add(ParagraphStyle(name="CustomHeaderTitle", fontSize=26, leading=30, textColor=colors.white, alignment=1))
styles.add(ParagraphStyle(name="CustomSectionTitle", fontSize=16, leading=20, textColor=colors.HexColor("#1E3A8A"), spaceAfter=10))
styles.add(ParagraphStyle(name="CustomBody", fontSize=11, leading=15, textColor=colors.HexColor("#1E293B")))
styles.add(ParagraphStyle(name="CustomHighlight", fontSize=12, leading=16, textColor=colors.HexColor("#2563EB")))

# ---------- Page Header ----------
def page_header(canvas, doc):
    canvas.saveState()
    width, height = landscape(A4)
    canvas.setFillColor(colors.HexColor("#1E3A8A"))
    canvas.rect(0, height - 45, width, 45, fill=1, stroke=0)
    canvas.setFont("Helvetica-Bold", 14)
    canvas.setFillColor(colors.white)
    canvas.drawString(40, height - 30, "U23 Player Grading System — Mid America")
    canvas.restoreState()

# ---------- Document ----------
doc = SimpleDocTemplate(
    PDF_PATH,
    pagesize=landscape(A4),
    rightMargin=40, leftMargin=40, topMargin=60, bottomMargin=40
)
elements = []

# ---------- Title Slide ----------
elements.append(Spacer(1, 80))
elements.append(Paragraph("U23 Player Grading System", styles["CustomHeaderTitle"]))
elements.append(Spacer(1, 10))
elements.append(Paragraph("Multi-League Report • Local Edition", styles["CustomHighlight"]))
elements.append(Spacer(1, 25))
elements.append(Paragraph("Filters Applied:", styles["CustomSectionTitle"]))
elements.append(Paragraph("• Age ≤ 23 • ≥ 900 Minutes • Non–Big 5 Leagues", styles["CustomBody"]))
elements.append(Spacer(1, 15))
elements.append(Paragraph(f"Total Players Analyzed: <b>{len(combined):,}</b>", styles["CustomBody"]))
elements.append(Paragraph(f"Filtered U23 Non–Big 5 Players: <b>{len(filtered):,}</b>", styles["CustomBody"]))
elements.append(Spacer(1, 70))
elements.append(Paragraph("Produced by: Automated Analytics Engine ⚙️", styles["CustomHighlight"]))
elements.append(PageBreak())

# ---------- League-Level Summary ----------
league_summary = (
    filtered.groupby("League")[["OFP_20_80", "Impact_20_80"]]
    .mean().round(1).sort_values("OFP_20_80", ascending=False)
)
elements.append(Paragraph("League-Level Averages", styles["CustomSectionTitle"]))
data = [["League", "Avg OFP", "Avg Impact"]] + league_summary.reset_index().values.tolist()
table = Table(data, colWidths=[240, 100, 100])
table.setStyle(TableStyle([
    ("BACKGROUND", (0,0), (-1,0), colors.HexColor("#1E3A8A")),
    ("TEXTCOLOR", (0,0), (-1,0), colors.white),
    ("ALIGN", (0,0), (-1,-1), "CENTER"),
    ("FONTNAME", (0,0), (-1,0), "Helvetica-Bold"),
    ("FONTSIZE", (0,0), (-1,-1), 10),
    ("GRID", (0,0), (-1,-1), 0.25, colors.grey),
    ("ROWBACKGROUNDS", (0,1), (-1,-1), [colors.whitesmoke, colors.lightgrey]),
]))
elements.append(table)
elements.append(PageBreak())

# ---------- Positional Distribution Chart ----------
pos_mix = filtered["PosGroup"].value_counts(normalize=True).mul(100).round(1)
fig, ax = plt.subplots(figsize=(8,5))
colors_list = ["#2563EB", "#1E40AF", "#60A5FA", "#93C5FD"]
ax.pie(pos_mix, labels=pos_mix.index, autopct='%1.1f%%', startangle=90, colors=colors_list)
ax.set_title("Positional Distribution (U23 Non–Big 5)", fontsize=14, color="#1E3A8A")
plt.tight_layout()
pie_path = os.path.join(FOLDER_PATH, "PositionMix.png")
plt.savefig(pie_path, dpi=300)
plt.close()

elements.append(Paragraph("Squad Composition Overview", styles["CustomSectionTitle"]))
# fill most of the slide width
elements.append(Image(pie_path, width=9.5*inch, height=5.5*inch))
elements.append(PageBreak())

# ---------- Top 25 Visuals ----------
for title, img_name in [
    ("Top 25 U23 Players by OFP", "Top25_OFP.png"),
    ("Top 25 U23 Players by Impact", "Top25_Impact.png")
]:
    img_path = os.path.join(FOLDER_PATH, img_name)
    if os.path.exists(img_path):
        elements.append(Paragraph(title, styles["CustomSectionTitle"]))
        # full-bleed chart fitting slide width
        elements.append(Image(img_path, width=10.5*inch, height=6.0*inch))
        elements.append(PageBreak())

# ---------- Top Performers Tables ----------
def add_table_section(title, df):
    elements.append(Paragraph(title, styles["CustomSectionTitle"]))
    cols = ["Player", "Team within selected timeframe", "League", "Age", "Position", "OFP_20_80", "Impact_20_80"]
    data = [cols] + df[cols].head(25).values.tolist()
    table = Table(data, repeatRows=1, colWidths=[140, 180, 120, 40, 60, 60, 60])
    table.setStyle(TableStyle([
        ("BACKGROUND", (0,0), (-1,0), colors.HexColor("#1E3A8A")),
        ("TEXTCOLOR", (0,0), (-1,0), colors.white),
        ("ALIGN", (3,1), (-1,-1), "CENTER"),
        ("GRID", (0,0), (-1,-1), 0.25, colors.grey),
        ("FONTNAME", (0,0), (-1,0), "Helvetica-Bold"),
        ("ROWBACKGROUNDS", (0,1), (-1,-1), [colors.whitesmoke, colors.lightgrey]),
    ]))
    elements.append(table)
    elements.append(Spacer(1, 25))

add_table_section("🏅 Top 15% OFP Performers", top15_ofp)
elements.append(PageBreak())
add_table_section("⚡ Top 15% Impact Performers", top15_impact)

# ---------- Build PDF ----------
doc.build(elements, onFirstPage=page_header, onLaterPages=page_header)
print(f"🎨 Presentation PDF report saved → {PDF_PATH}")


🎨 Presentation PDF report saved → /Users/user/Downloads/Mid America/U23 Mid America.pdf
